In [ ]:
from openai import OpenAI
from openai.types.chat import ChatCompletion
from datetime import datetime

# 课程案例给的API，chatfire转
BASE_URL: str = "https://api.chatfire.cn/v1"
API_KEY: str = "sk-6vsg4LCMHd9q4o8ZUKQzMLVrjVMgL21opUkr8bRR33gBrECJ"

# timestamp = datetime.now().strftime("%Y%m%d_%H%M")  
timestamp = 2  # 这个是为了保存多版本，调试用。
client = OpenAI(base_url=BASE_URL, api_key=API_KEY)

USER_REQUIREMENT = "我要一个ATM系统，支持用户登录、查询余额、取款、存款、转账、修改密码。"

# Step 1: 需求分析 Agent
def analyze_requirement(user_input):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": (
                "你是一名资深的软件需求分析专家，擅长将自然语言描述转化为结构化系统需求。"
                "请从用户输入中识别出以下要素：\n"
                "1. 系统名称\n2. 用户角色\n3. 核心功能点（动词+对象形式，例如：查询余额）\n"
                "请用以下格式输出：\n"
                "系统名称：\n用户角色：\n功能列表：\n- 功能1\n- 功能2\n..."
            )},
            {"role": "user", "content": user_input}
        ]
    )
    return response.choices[0].message.content

# Step 2: UML建模 Agent（支持上下文反馈）
def generate_uml(requirement_model, previous_output=None, feedback=None):
    user_prompt = f"""以下是系统的结构化功能需求：{requirement_model}

            请你根据上述需求生成 ATM 系统的 UML 类图（Class Diagram）和顺序图（Sequence Diagram）。
            输出格式如下：

            【类图】
            - 类名1
              - 属性: ...
              - 方法: ...
            - 类名2
              - 属性: ...
              - 方法: ...

            【顺序图】
            请列出典型操作（如“取款”）下的对象交互流程，每一步清晰表述消息发送方、接收方与内容。

            """

    if previous_output:
        user_prompt += f"\n以下是上一次的建模输出内容，请你参考并改进：\n{previous_output}\n"
    if feedback:
        user_prompt += f"\n评估员给出的修改建议如下，请特别注意改进：\n{feedback}\n"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "你是一名系统建模专家，请生成ATM系统的类图（类、属性、方法）和顺序图（消息交互）。"},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content, user_prompt

# Step 2.5: 质量评估 Agent
def evaluate_uml(uml_output, req_output,force_fail_first=False, attempt=1):
    eval_prompt = f"""
            你是一位非常严格的 UML 建模质量评估专家，你的任务是审查下面的 UML 类图与顺序图的建模输出是否合格。

            评估要求如下（必须全部满足才能通过）：
            1. UML 类图中必须至少包含 3 个类，每个类应包含不少于 2 个属性和 2 个方法。
            2. 所有类方法必须能涵盖用户需求中所有核心功能点。
            3. 类与类之间必须存在清晰的关系（如关联、依赖、继承等），并有明确说明。
            4. 顺序图必须描述上述所有的核心业务流程，且每一步要说明：消息发送方、接收方、消息内容。
            5. 图中不得出现“某某类可能做某事”这类不确定性语句。
            6. 类命名与方法命名必须专业、清晰、符合软件工程规范。
            7. 输出不得冗长废话，必须是结构化的 UML 建模内容。
            8. 最后要检查是否有遗漏，用户的需求功能如下：
            {req_output}
            

            请严格执行以上标准。即便小问题也不允许通过。以下是要评估的内容：
            以下是被评估内容：
            \"\"\"{uml_output}\"\"\"

            请仅用以下格式回复：

            是否通过：仅回答 通过 或者 不通过
            存在问题：
            - 类图缺少XXX
            - 顺序图未体现XXX
            改进建议：
            - 添加XXX类及其方法
            - 丰富顺序图中的消息交互
            """
    

    if force_fail_first and attempt == 1:
        eval_prompt += "\n注意：本轮为首次评估，必须返回“不通过”。\n"

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "你是一名严谨的UML建模评估员"},
            {"role": "user", "content": eval_prompt}
        ]
    )
    return response.choices[0].message.content,eval_prompt

# Step 3: 原型生成 Agent
def generate_prototype(uml_output):
    response = client.chat.completions.create(
        model="gpt-4o",
#         messages=[
#             {"role": "system", "content": "你是一名前端开发专家，请根据提供的UML需求和操作来生成相应的HTML页面原型代码"},
#             {"role": "user", "content": uml_output}
#         ]
        messages=[
            {"role": "system", "content": (
                "你是一名前端开发专家，请根据以下 UML 类图与顺序图，"
                "生成一个 ATM 系统的 HTML 页面原型代码。\n"
                "页面应包含：登录界面、账户操作区（取款、查询、转账等）、输出提示区等。\n"
                "请使用结构良好的 HTML + 内联 CSS。页面布局应清晰、用户友好。\n"
                "仅返回 HTML 代码部分（不要额外说明），便于直接保存为 .html 使用。"
                "注意最终要预览下界面的效果，不要有一些干扰的文字出现，要生成一个直观的html界面"
            )},
            {"role": "user", "content": uml_output}
        ]
    )
    return response.choices[0].message.content

# 主流程
def main():
    print("Step 1: 分析用户需求...")
    requirement_model = analyze_requirement(USER_REQUIREMENT)
    print("需求模型输出：\n", requirement_model)

    previous_output = None
    eval_feedback = None

    log_lines = []
    uml_final_result = None

    for attempt in range(1, 6):
        print(f"\nStep 2: 第 {attempt} 次尝试生成 UML 模型...")
        uml_output, uml_messages = generate_uml(requirement_model, previous_output, eval_feedback)
        print("UML建模输出：\n", uml_output)

        eval_result, eval_messages = evaluate_uml(uml_output,requirement_model, force_fail_first=True, attempt=attempt)
        print("评估结果：\n", eval_result)

        # 记录本轮日志
        log_lines.append(f"===== 第{attempt}轮 UML Agent 输入 =====\n{uml_messages}\n")
        log_lines.append(f"===== 第{attempt}轮 UML Agent 输出 =====\n{uml_output}\n")
        log_lines.append(f"===== 第{attempt}轮 Evaluation Agent 输入 =====\n{eval_messages}\n")
        log_lines.append(f"===== 第{attempt}轮 Evaluation Agent 输出 =====\n{eval_result}\n\n")

        if "不通过" not in eval_result:
            break
        else:
            print("模型质量不达标，准备重试...")
            previous_output = uml_output
            eval_feedback = eval_result
            time.sleep(2)
            
    uml_final_result = uml_output

    print("\nStep 3: 生成 HTML 原型页面...")
    prototype_html = generate_prototype(uml_final_result)
    print(prototype_html)
    # 保存最终 UML 输出

    with open(f"uml_{timestamp}.md", "w", encoding="utf-8") as f:
        f.write(uml_final_result)

    with open(f"prototype_{timestamp}.html", "w", encoding="utf-8") as f:
        f.write(prototype_html)
    print("\n生成完毕，HTML 已保存为 prototype.html")

    # 保存日志
    with open(f"log_{timestamp}.txt", "w", encoding="utf-8") as f:
        f.writelines(log_lines)

if __name__ == "__main__":
    main()